#### Define 'EPOCHS' (total epochs) and 'EP_INT' (interval epochs) and then run from the starting...

In [3]:
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext import  vocab,data
from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator, TabularDataset
from torchvision.transforms import ToTensor

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import os, csv, sys, random, re, time, math, spacy, nltk

from PIL import Image
from numpy.random import RandomState
from tensorboardX import SummaryWriter
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [4]:
nlp = spacy.load('en_core_web_sm')

#Define the logger
#log_writer_train = SummaryWriter('TBlogs/train/')
#log_writer_val = SummaryWriter('TBlogs/val/')
#log_writer_test = SummaryWriter('TBlogs/test/')

log_writer = SummaryWriter('TBlogs/')

In [5]:
EPOCHS = 5     # Total epochs to train for
EP_INT = 1     # In the intervals of 'EP_INT' epochs
CLIP = 1

NF=1000        # For ResNet/VGG Feature Extraction
#NF = 36*2048  # For Faster RCNN Feature Extraction using Object Detection

In [6]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [7]:
dataset=pd.read_csv(r"train_data.csv")

In [8]:
dataset.head(-1)

,src,trg,img_path,img_id
0,video a bizarre conspiracy theory has surged i...,i take news with a pinch of salt so should eve...,/home/puneet/code/Multimodal Feedback/data/781...,7817
1,of the army which is increasingly dependent on...,honestly i never thought to earn as much as ha...,/home/puneet/code/Multimodal Feedback/data/381...,3817
2,however on thursday he admitted that he had re...,but he attacked so much poland accused defamed...,/home/puneet/code/Multimodal Feedback/data/130...,1306
3,the pentagon said thursday that the us militar...,great news,/home/puneet/code/Multimodal Feedback/data/651...,6514
4,washington pressure is growing among president...,more good news please,/home/puneet/code/Multimodal Feedback/data/230...,2309
...,...,...,...,...
58336,kabul afghanistan the taliban ambushed a peace...,i always for the of reeducation no hope= misle...,/home/puneet/code/Multimodal Feedback/data/669...,6697
58337,file photo the olympic rings are illuminated i...,the crazy just never seems to stop in florida ...,/home/puneet/code/Multimodal Feedback/data/541...,5417
58338,saudi arabia has sentenced five people to deat...,five lives to save face for the prince,/home/puneet/code/Multimodal Feedback/data/627...,6271
58339,acting manatee county administrator dr scott h...,floridas republican governors have been availa...,/home/puneet/code/Multimodal Feedback/data/935...,9358


In [9]:
def tokenize_text(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in nlp.tokenizer(text)]

In [10]:
ID = data.Field(sequential=False,use_vocab=False)
SRC = Field(tokenize = tokenize_text, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True,
            include_lengths = True)
TRG = Field(tokenize = tokenize_text, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True,
            )
datafields=[('src', SRC), ('trg', TRG),('img_path',None),('img_id',ID)]
datafields

[('src', <torchtext.data.field.Field at 0x7f17fbff7290>),
 ('trg', <torchtext.data.field.Field at 0x7f17fbff7310>),
 ('img_path', None),
 ('img_id', <torchtext.data.field.Field at 0x7f17fbff72d0>)]

In [11]:
#df = pd.read_csv("data.csv")

#rng = RandomState()
#train_data = df.sample(frac=0.80, random_state=rng)
#val_data = df.loc[~df.index.isin(train_data.index)]

##cols= ["src", "trg", "img_path", "img_id"]
#train_data.to_csv('train_data.csv', index= False) #columns=cols
#val_data.to_csv('val_data.csv', index= False) #columns=cols

In [ ]:
train_data, val_data = data.TabularDataset.splits(path=r"",train="train_data.csv", validation="val_data.csv", format='csv', skip_header=True, fields=datafields)

In [ ]:
print(len(train_data))
print(len(val_data))

In [ ]:
#print(vars(train_data.examples[0]))

In [ ]:
sys.maxsize

In [ ]:
SRC.build_vocab(train_data, vectors="glove.6B.100d")
TRG.build_vocab(train_data, vectors="glove.6B.100d")
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=1)

In [ ]:
print(f"Unique tokens in target (en) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

In [ ]:
BATCH_SIZE = 4 #16

train_iterator, x_iterator = BucketIterator.splits(
    (train_data, train_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    sort_key = lambda x : len(x.src), 
    device = device)

valid_iterator, y_iterator = BucketIterator.splits(
    (val_data, val_data), 
    batch_size = BATCH_SIZE, 
    sort_within_batch = True,
    sort_key = lambda x : len(x.src),
    device = device)

#valid_iterator, y_iterator = BucketIterator.splits(
#    (val_data, val_data), 
#    batch_size = BATCH_SIZE, 
#    device = device)

In [ ]:
print(len(train_iterator))
print(len(valid_iterator))

In [ ]:
for i, batch in enumerate(valid_iterator):
  print(i,batch)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.fc = nn.Linear(hid_dim * 2, hid_dim)
        self.vis_fc=nn.Linear(NF, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, visual_features):
        
        vis_features= torch.tanh(self.vis_fc(visual_features))
            
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, text_hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #text_hidden = [n layers * n directions, batch size, hid dim]
        vis_features=vis_features.unsqueeze(0)
        hidden = torch.tanh(self.fc(torch.cat((text_hidden,vis_features), 2)))
        
        #outputs are always from the top hidden layer
        
        return hidden

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, visual_features, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src,visual_features)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_HID_DIM = 125
ENC_EMB_DIM = 64
DEC_EMB_DIM = 64
HID_DIM = 128
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
INPUT_DIM

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src,src_len = batch.src
        trg = batch.trg
        img_id=batch.img_id
        
        x=img_id.cpu().numpy()
        y=len(x)
        #visual_features=torch.empty(y,4096).cuda()
        visual_features=torch.empty(y,NF).cuda()
        df=pd.read_csv(r"visual_features_resnet.csv")
        for i in range(y):
            q=df[str(x[i])].to_numpy()
            a=torch.from_numpy(q).unsqueeze(0)
            visual_features[i]=a
        
        optimizer.zero_grad()
        
        output = model(src,visual_features, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src,src_len = batch.src
            trg = batch.trg
            img_id=batch.img_id
        
            x=img_id.cpu().numpy()
            y=len(x)
            #visual_features=torch.empty(y,4096).cuda()
            visual_features=torch.empty(y,NF).cuda()
            df=pd.read_csv(r"visual_features_resnet.csv")
            for i in range(y):
                q=df[str(x[i])].to_numpy()
                a=torch.from_numpy(q).unsqueeze(0)
                visual_features[i]=a

            output = model(src, visual_features, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
def load_checkpt(model, optimizer, chpt_file):
    start_epoch = 0
    if (os.path.exists(chpt_file)):
        print("=> loading checkpoint '{}'".format(chpt_file))
        checkpoint = torch.load(chpt_file)
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        start_epoch = checkpoint['epoch']
        
        print("=> loaded checkpoint '{}' (epoch {})".format(chpt_file, checkpoint['epoch']))
        
    else:
        print("=> Checkpoint NOT found '{}'".format(chpt_file))
    return model, optimizer, start_epoch

In [ ]:
def translate_sentence(sentence, src_field, trg_field, img_id, path, model, device, max_len = 50):

    model.eval()
        
    if isinstance(sentence, str):
        nlp = spacy.load('en')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    src_len = torch.LongTensor([len(src_indexes)]).to(device)
    
    df=pd.read_csv(path)
    visual_features=torch.empty(1,NF).cuda()
#    visual_features=torch.empty(1,4096).cuda()
    q=df[str(img_id)].to_numpy()
    a=torch.from_numpy(q).unsqueeze(0)
    visual_features[0]=a

    with torch.no_grad():
        hidden = model.encoder(src_tensor,visual_features)#, src_len)
        
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]
    
    for i in range(max_len):
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)          
        with torch.no_grad():
            output, hidden = model.decoder(trg_tensor, hidden, hidden)

        pred_token = output.argmax(1).item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:]

In [ ]:
'''text_idx = 25                #renamed from 'example_idx'
image_idx="9"

path=r"visual_features.csv"

src = vars(train_data.examples[text_idx])['src']
trg = vars(train_data.examples[text_idx])['trg']
#img = vars(train_data.examples[text_idx])['img_path']

translation = translate_sentence(src, SRC, TRG, image_idx,path, model, device)

#print(f'src = {src}\n')
print(f'Ground-truth Comment')
print( '--------------------')
print(f'trg = {trg}')

print(f'\n\nPredicted Feedback (Feedback)')
print( '-----------------------------')
    
print(f'{translation}')
'''

In [ ]:
### Spice & Meteor
#### (Hitting erorr in py3, need to run in py2...) tried switching the kernel to py2 and runnign here but T_EPOCHS is cleared off...
#### 1. Go to '/home/puneet/code/EvalMetrics_py2'
#### 2. Copy 'nEp_test_comments.csv' & 'nEp_test_feedbacks.csv'
#### 3. Rename as 'test_comments.csv' & 'test_feedbacks.csv'
#### 4. Run eval.py with 'py2' env
#### 5. Rename the generated 'scores.txt' file  

In [ ]:
def quant_eval(model, optimizer, start_epoch, chpt_file):
    from pycocoevalcap.bleu.bleu import Bleu
    from pycocoevalcap.cider.cider import Cider
    from pycocoevalcap.meteor.meteor import Meteor
    from pycocoevalcap.rouge.rouge import Rouge
    from pycocoevalcap.spice.spice import Spice
    import os, json, csv
    
    model, optimizer, start_epoch = load_checkpt(model, optimizer, chpt_file)
    T_EPOCHS = start_epoch + EP_INT    
    

    #print('Download Stanford models... Run once!')
    os.system("sh get_stanford_models.sh")

    with open('temp/'+str(T_EPOCHS)+'Ep_test_comments.csv',"r") as f: 
            reader = csv.reader(f)
            gts = {rows[0]:rows[1:] for rows in reader}
            #print(mydict) #prints with single quotes
            #print (json.dumps(mydict)) #prints with double quotes

    with open('temp/'+str(T_EPOCHS)+'Ep_test_feedbacks.csv',"r") as g: 
            reader = csv.reader(g)
            res = {rows[0]:rows[1:] for rows in reader}
            #print(json.dumps(mydict))

    '''with open('temp/test_comments.json', 'r') as file:
        gts = json.load(file)
    with open('temp/test_feedbacks.json', 'r') as file:
        res = json.load(file)
    '''

    def bleu():
        scorer = Bleu(n=4)
        score, scores = scorer.compute_score(gts, res)
        return score


    def cider():
        scorer = Cider()
        (score, scores) = scorer.compute_score(gts, res)
        return score

    def rouge():
        scorer = Rouge()
        score, scores = scorer.compute_score(gts, res)
        return score

    #bgts = gts[0].encode(encoding='UTF-8')
    #bres = res[0].encode(encoding='UTF-8')

    def spice():
        scorer = Spice()
        #print(gts, res)
        score, scores = scorer.compute_score(gts, res)
        return score

    def meteor():
        scorer = Meteor()
        #print(gts, res)
        score, scores = scorer.compute_score(bgts, bres)
        return score    
    s_cider=cider()
    s_rouge=rouge()
    s_bleu=bleu()
    #s_spice=spice()#
    #s_meteor=meteor()#
    
    print('\n----------------------\nbleu = %s' %s_bleu )
    print('cider = %s' %s_cider )
    print('rouge = %s' %s_rouge )
    #print('spice = %s' %s_spice )
    #print('meteor = %s' %s_meteor )
    
    b=" ".join(str(x) for x in s_bleu)
    print('\n----------------------')
    f = open('scores.txt', 'w') 
    f.write("\ncider: %f" % s_cider)
    f.write("\nrouge: %f" % s_rouge)
    #f.write("\nspice: %f" % s_spice)
    #f.write("\nmeteor: %f" % s_meteor)
    f.write("\nbleu :")
    f.write(b)
    f.close()
    
    #print(str(T_EPOCHS))
    #Log with Tensorboard: Eval metrics
    log_writer.add_text(str(T_EPOCHS)+'Ep=>Metrics/cider', str(s_cider))
    log_writer.add_text(str(T_EPOCHS)+'Ep=>Metrics/rouge', str(s_rouge))
    #log_writer.add_text(str(T_EPOCHS)+'Ep=>Metrics/spice', str(s_spice))
    #log_writer.add_text(str(T_EPOCHS)+'Ep=>Metrics/meteor', str(s_meteor))
    log_writer.add_text(str(T_EPOCHS)+'Ep=>Metrics/bleu-1', str(s_bleu[0]))
    log_writer.add_text(str(T_EPOCHS)+'Ep=>Metrics/bleu-2', str(s_bleu[1]))
    log_writer.add_text(str(T_EPOCHS)+'Ep=>Metrics/bleu-3', str(s_bleu[2]))
    log_writer.add_text(str(T_EPOCHS)+'Ep=>Metrics/bleu-4', str(s_bleu[3]))

In [ ]:
def qual_eval(model, optimizer, start_epoch, chpt_file):
    model, optimizer, start_epoch = load_checkpt(model, optimizer, chpt_file)
    T_EPOCHS = start_epoch + EP_INT
    
    # Save the predicted Feedbacks in CSV file
    # Log with Tensorboard: Text, Comment, Image and Feedback
    test_pred=[]
    #test_df=pd.read_csv("temp/test_data.csv")  
    test_df=pd.read_csv("temp/test_data1.csv") 
    path=r"visual_features_resnet.csv"      
    length=test_df.shape[0]
    images = []

    #print(length)
    for i in range(length):
        src=test_df['src'][i]
        trg=test_df['trg'][i]
        img_path=test_df['img_path'][i]
        image_idx=test_df['img_id'][i]
        translation, attention = translate_sentence(src, SRC, TRG, image_idx, path, model, device)

        if not translation:
            translation="*empty*"

        #Untokenization    
        translation1=translation[0:(len(translation)-1)]    
        translation2 = TreebankWordDetokenizer().detokenize(translation1)
        test_pred.append(str(translation2))

        image = Image.open(img_path)
        image = ToTensor()(image)   
        #images.append(image)
        
        #print(str(T_EPOCHS))
        if (i%10==0): 
            #Log with Tensorboard: Text, Comment, Image and Feedback
            log_writer.add_text(str(T_EPOCHS)+'Ep=>Ground-truth Comment of Sample/'+str(i+1), str(trg))
            log_writer.add_text(str(T_EPOCHS)+'Ep=>News Text of Sample/'+str(i+1), str(src))#, i+1)
            log_writer.add_text(str(T_EPOCHS)+'Ep=>Predicted Feedback of Sample/'+str(i+1), str(translation))
            #log_writer.add_image('Image', image, i+1)
            #image_grid = torchvision.utils.make_grid(images)
            log_writer.add_image(str(T_EPOCHS)+'Ep:Image of Sample/'+str(i+1), image)        

    with open('temp/'+str(T_EPOCHS)+'Ep_test_results.csv', 'w'): 
        pass
    with open('temp/'+str(T_EPOCHS)+'Ep_test_comments.csv', 'w'): 
        pass
    with open('temp/'+str(T_EPOCHS)+'Ep_test_feedbacks.csv', 'w'): 
        pass

    test_df["pred"] = test_pred 
    test_df.to_csv('temp/'+str(T_EPOCHS)+'Ep_test_results.csv', index= False)
    test_df.to_csv('temp/'+str(T_EPOCHS)+'Ep_test_comments.csv', index= True, columns=["trg"]) #index -> "key": value -> ["trg/pred"]
    test_df.to_csv('temp/'+str(T_EPOCHS)+'Ep_test_feedbacks.csv', index= True, columns=["pred"])

    #Re-open and save with new column names
    df = pd.read_csv('temp/'+str(T_EPOCHS)+'Ep_test_comments.csv')
    df.columns = ['id', 'comment']
    df.to_csv('temp/'+str(T_EPOCHS)+'Ep_test_comments.csv', index= False)

    df = pd.read_csv('temp/'+str(T_EPOCHS)+'Ep_test_feedbacks.csv')
    df.columns = ['id', 'feedback']
    df.to_csv('temp/'+str(T_EPOCHS)+'Ep_test_feedbacks.csv', index= False)
    
    print('tensorboard --logdir "/home/puneet/code/Multimodal Feedback/TBlogs"\n---')

In [ ]:
def interval_train(model, optimizer, start_epoch, chpt_file):
    model, optimizer, start_epoch = load_checkpt(model, optimizer, chpt_file)
    T_EPOCHS = start_epoch + EP_INT

    print('Already trained for',start_epoch, 'epochs. Training now for', EP_INT, 'more')

    best_valid_loss = float('inf')
    cur_best_train_loss = float('inf')

    #for epoch in range(EP_INT):
    for epoch in range(start_epoch, T_EPOCHS):    
        start_time = time.time()

        train_loss = train(model, train_iterator, optimizer, criterion, CLIP)#, log_writer_train)
        valid_loss = evaluate(model, valid_iterator, criterion)#, log_writer_val)

        #Log with Tensorboard: Loss & PPL for train & val 
        log_writer.add_scalar('Train/Loss',float(train_loss), epoch+1)
        log_writer.add_scalar('Train/PPL', float(math.exp(train_loss)), epoch+1)    
        log_writer.add_scalar('Val/Loss',float(valid_loss), epoch+1)
        log_writer.add_scalar('Val/PPL', float(math.exp(valid_loss)), epoch+1)

        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        #if valid_loss < best_valid_loss:
        #    best_valid_loss = valid_loss
        #if train_loss < cur_best_train_loss:
        #    cur_best_train_loss = train_loss

        state = {'epoch': T_EPOCHS, 'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict(), 'loss': train_loss}
        torch.save(state, chpt_file)

        print(f'\nEpoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

    torch.cuda.empty_cache()

In [ ]:
chpt_file = '/home/puneet/code/Multimodal Feedback/checkpoints/baseline1.pt'
model, optimizer, start_epoch = load_checkpt(model, optimizer, chpt_file)
T_EPOCHS = start_epoch + EP_INT
    
for i in range(EPOCHS):
    if (i%EP_INT==0):
        interval_train(model, optimizer, start_epoch, chpt_file)
        print('\nEvaluation\n----------------------')
        qual_eval(model, optimizer, start_epoch, chpt_file)
        quant_eval(model, optimizer, start_epoch, chpt_file)